In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import spacy
import seaborn as sns
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
!python -m spacy download el_core_news_sm

In [ ]:
nlp = spacy.load('el_core_news_sm')

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), min_df=0.01, max_df=0.95)

In [ ]:
start_page = 0
end_page = 17

start_url = 'https://www.avgi.gr/search?keywords=%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82&_wrapper_format=html&page='

test_url = start_url + str(start_page)

test_url

'https://www.avgi.gr/search?keywords=%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82&_wrapper_format=html&page=0'

In [ ]:
response = requests.get(test_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [ ]:
teaser_articles_list = doc.find('div', {'class': 'horizontal-teasers'}).find_all('article')

In [ ]:
teaser_articles_list

In [ ]:
link = teaser_articles_list[0].find('a')['href']

link

'/diethni/454475_o-atsalakotos-kyrios-royte'

In [ ]:
main_url = 'https://www.avgi.gr'
article_link = main_url + link
print(article_link)

https://www.avgi.gr/diethni/454475_o-atsalakotos-kyrios-royte


In [ ]:
teasers_list = []

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo teaser_articles_list***
  teaser_articles_list = doc.find('div', {'class': 'horizontal-teasers'}).find_all('article')


  for teaser in teaser_articles_list:
      story_dict = {}
      try:

        #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στο link, ΜΕΤΑ το teaser
        link = teaser.find('a')['href']


        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      teasers_list.append(story_dict)
  time.sleep(1)

In [ ]:
avgi_teasers_df = pd.DataFrame(teasers_list)
avgi_teasers_df

,url
0,https://www.avgi.gr/diethni/454475_o-atsalakot...
1,https://www.avgi.gr/tehnes/450809_peri-bibliot...
2,https://www.avgi.gr/politiki/446755_paraskinio...
3,https://www.avgi.gr/diethni/439671_katadikasti...
4,https://www.avgi.gr/diethni/436979_antiembolia...
...,...
168,https://www.avgi.gr/arheio/364871_antiembolias...
169,https://www.avgi.gr/arheio/363853_ypohreotikos...
170,https://www.avgi.gr/entheta/prisma/287180_embo...
171,https://www.avgi.gr/koinonia/284812_o-arbaniti...


In [ ]:
avgi_teasers_df.to_csv('/content/temp/avgi_teasers.csv', index=False)

In [ ]:
article_url = 'https://www.avgi.gr/diethni/454475_o-atsalakotos-kyrios-royte'

In [ ]:
response = requests.get(article_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [ ]:
article = doc.find('article', {'class': 'article'})

In [ ]:
section = doc.find('div', {'class': 'article__category'}).find('a').text
section

'ΔΙΕΘΝΗ'

In [ ]:
title = article.find('section', {'class': 'article__top'}).find('h1', {'class': 'double-title'}).text
title

' Ο ατσαλάκωτος κύριος Ρούτε'

In [ ]:
date = article.find('time', {'class': 'default-date'})['datetime']
date

'2023-07-11T12:01:03+03:00'

In [ ]:
author = article.find('a', {'class', "article__author"}).span.text
author

'Κώστας Αργυρός'

In [ ]:
paragraphs = article.find_all('p')
paragraphs

In [ ]:
p_texts_list = []                      # δημιουργία κενής λίστας για να αποθηκεύσουμε το text κάθε παραγράφου
paragraphs = article.find_all('p')     # λίστα με όλες τις παραγράφους
for p in paragraphs:                   # για κάθε μια από τις παραγράφους στη λίστα
  p_texts_list.append(p.text)          # πάρε το .text και πρόσθεσέ το στη λίστα με τα texts των παραγράφων
full_text = ' '.join(p_texts_list)     # συνένωση των texts των παραγράφων
full_text = "".join(full_text.splitlines())
full_text

In [ ]:
full_articles_list = []

for article_url in avgi_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo article που βρήκατε παραπάνω***
  article = doc.find('article', {'class': 'article'})

  full_article_dict = {}

  # website
  full_article_dict['site'] = "avgi.gr"

  # url
  full_article_dict['url'] = article_url

  #section
  try:
    section = doc.find('div', {'class': 'article__category'}).find('a').text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  #title
  try:
    title = article.find('section', {'class': 'article__top'}).find('h1', {'class': 'double-title'}).text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  # date
  try:
    date = article.find('time', {'class': 'default-date'})['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass

  # author
  try:
    author = article.find('a', {'class', "article__author"}).span.text
    full_article_dict['author'] = author
  except:
    full_article_dict['author'] = ''
    pass

  # full_text
  try:
    p_texts_list = []
    paragraphs = article.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
  time.sleep(1)

In [ ]:
avgi_full_articles_df = pd.DataFrame(full_articles_list)
avgi_full_articles_df

,site,url,section,title,date,author,full_text
0,avgi.gr,https://www.avgi.gr/diethni/454475_o-atsalakot...,ΔΙΕΘΝΗ,Ο ατσαλάκωτος κύριος Ρούτε,2023-07-11T12:01:03+03:00,Κώστας Αργυρός,Παρέμεινε στο τιμόνι των ολλανδικών κυβερνήσεω...
1,avgi.gr,https://www.avgi.gr/tehnes/450809_peri-bibliot...,ΤΕΧΝΕΣ,Περί βιβλιοθηκών... συλλογές,2023-06-12T17:30:11+03:00,Καλλίμαχος,Εχουμε παρουσιάσει από τη στήλη τους περασμένο...
2,avgi.gr,https://www.avgi.gr/politiki/446755_paraskinio...,ΠΟΛΙΤΙΚΗ,Εκλογές 2023 / Το παρασκήνιο της διάσπασης τη...,2023-05-01T15:29:10+03:00,Γιάννης Ανδρουλιδάκης,"Στις 20 Φεβρουαρίου 1933, στην Καγκελαρία του ..."
3,avgi.gr,https://www.avgi.gr/diethni/439671_katadikasti...,ΔΙΕΘΝΗ,Γαλλία / Καταδικάστηκαν δύο γυναίκες που διέδ...,2023-02-15T23:23:05+02:00,Αυγή Newsroom,"Δικαστήριο στη Γαλλία καταδίκασε δύο γυναίκες,..."
4,avgi.gr,https://www.avgi.gr/diethni/436979_antiembolia...,ΔΙΕΘΝΗ,Νέα Ζηλανδία / Αντιεμβολιαστές και λόμπι των ...,2023-01-21T16:40:31+02:00,Νίκος Κυριακίδης,Από την πρώτη κιόλας στιγμή της ανάδυσής της σ...
...,...,...,...,...,...,...,...
168,avgi.gr,https://www.avgi.gr/arheio/364871_antiembolias...,ΑΡΧΕΙΟ,"Αντιεμβολιαστές, ακροδεξιοί και συνωμοσιολόγο...",2020-08-29T09:48:00+03:00,Αυγή Newsroom,Νωρίτερα αυτή την εβδομάδα οι γερμανικές αρχές...
169,avgi.gr,https://www.avgi.gr/arheio/363853_ypohreotikos...,ΑΡΧΕΙΟ,Υποχρεωτικός θα είναι ο εμβολιασμός κατά του ...,2020-08-19T08:55:00+03:00,Αυγή Newsroom,Οι αρχές της Αυστραλίας αναμένεται να καταστήσ...
170,avgi.gr,https://www.avgi.gr/entheta/prisma/287180_embo...,ΠΡΙΣΜΑ,Εμβόλια και αντιεμβολιασμός,2018-09-29T19:00:00+03:00,Αυγή Newsroom,"Φαίνεται πως, μολονότι πολλές ασθένειες έχουν ..."
171,avgi.gr,https://www.avgi.gr/koinonia/284812_o-arbaniti...,ΚΟΙΝΩΝΙΑ,"Ο Αρβανίτης, οι γιατροί και τα εμβόλια",2018-09-06T19:00:00+03:00,Αυγή Newsroom,Σάλος στο Ίντερνετ και τα social media δημιουρ...


In [ ]:
avgi_full_articles_df.to_csv('/content/temp/avgi_full_articles.csv', index=False)